<a href="https://colab.research.google.com/github/NimishK26/Analytics-in-Practice---WBS/blob/main/PowerBI_RAG_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Environment Setup & Imports

In [ ]:
# Step 1: Environment Setup & Imports (run this first)
!pip install --upgrade \
    datasets \
    langchain \
    sentence-transformers \
    faiss-cpu \
    transformers \
    beautifulsoup4 \
    accelerate

import os
import glob
import re
import pickle
import numpy as np
import torch
import faiss
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig, AutoModelForCausalLM

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


Using device: cpu


# Step 2: Clone or Load Power BI Documentation

In [ ]:
!git clone https://github.com/MicrosoftDocs/powerbi-docs.git

Cloning into 'powerbi-docs'...
remote: Enumerating objects: 350052, done.
remote: Counting objects: 100% (691/691), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 350052 (delta 533), reused 549 (delta 460), pack-reused 349361 (from 3)
Receiving objects: 100% (350052/350052), 2.56 GiB | 26.61 MiB/s, done.
Resolving deltas: 100% (250238/250238), done.
Updating files: 100% (11021/11021), done.


In [ ]:
# List all items in the current working directory (including hidden)
!ls -a

# Recursively search for any folder named "powerbi-docs" (case-insensitive)
import os
for root, dirs, files in os.walk('.'):
    for d in dirs:
        if d.lower() == "powerbi-docs":
            print("Found at:", os.path.join(root, d))

.  ..  .config	powerbi-docs  sample_data
Found at: ./powerbi-docs
Found at: ./powerbi-docs/powerbi-docs


In [ ]:
# List contents of the inner powerbi-docs directory
!ls powerbi-docs/powerbi-docs


breadcrumb	   developer	 guided-learning   paginated-reports  visuals
collaborate-share  docfx.json	 includes	   report-server
connect-data	   enterprise	 index.yml	   support
consumer	   fundamentals  media		   transform-model
create-reports	   guidance	 natural-language  troubleshoot


In [ ]:
# Find every .md file under the powerbi-docs folder
!find powerbi-docs -type f -name "*.md" | head -n 20


powerbi-docs/README.md
powerbi-docs/powerbi-docs/paginated-reports/paginated-reports-edit-service.md
powerbi-docs/powerbi-docs/paginated-reports/web-authoring/get-started-paginated-formatted-table.md
powerbi-docs/powerbi-docs/paginated-reports/web-authoring/create-parameters.md
powerbi-docs/powerbi-docs/paginated-reports/web-authoring/paginated-formatted-table.md
powerbi-docs/powerbi-docs/paginated-reports/web-authoring/filters-paginated-service.md
powerbi-docs/powerbi-docs/paginated-reports/subreports-troubleshoot.md
powerbi-docs/powerbi-docs/paginated-reports/report-builder-planning-report.md
powerbi-docs/powerbi-docs/paginated-reports/troubleshoot-paginated-reports-execution-issues.md
powerbi-docs/powerbi-docs/paginated-reports/report-builder-url-parameters.md
powerbi-docs/powerbi-docs/paginated-reports/paginated-reports-diagnostics.md
powerbi-docs/powerbi-docs/paginated-reports/paginated-reports-embed.md
powerbi-docs/powerbi-docs/paginated-reports/paginated-reports-data-sources.md


In [ ]:

# 3.2: Helper to convert GitHub-style path → docs.microsoft.com URL
def md_path_to_url(md_path: str) -> str:
    """
    Given a path like
      'powerbi-docs/powerbi-docs/paginated-reports/web-authoring/create-parameters.md'
    Convert it to:
      'https://docs.microsoft.com/en-us/power-bi/paginated-reports/web-authoring/create-parameters'
    Steps:
      1. Remove 'powerbi-docs/powerbi-docs/' prefix and '.md' suffix
      2. Lowercase the remainder (docs use lowercase URLs)
      3. Prepend 'https://docs.microsoft.com/en-us/power-bi/'
    """
    # 1. Strip prefix and suffix
    rel = md_path.replace("powerbi-docs/powerbi-docs/", "").replace(".md", "")
    # 2. Lowercase
    rel = rel.lower()
    # 3. Prepend base URL
    return f"https://docs.microsoft.com/en-us/power-bi/{rel}"

# 3.3: Find all .md files under 'powerbi-docs/powerbi-docs'
md_files = glob.glob("powerbi-docs/powerbi-docs/**/*.md", recursive=True)
print(f"Found {len(md_files)} markdown files under powerbi-docs/powerbi-docs")

# 3.4: Read & extract metadata for each file
all_docs = []
for md_path in md_files:
    with open(md_path, "r", encoding="utf-8") as f:
        content = f.read()
    # Extract the first-level heading as title
    title_match = re.search(r"^# (.+)$", content, flags=re.MULTILINE)
    if title_match:
        title = title_match.group(1).strip()
    else:
        # Fallback: filename without extension
        title = os.path.basename(md_path).replace(".md", "")
    url = md_path_to_url(md_path)
    all_docs.append({
        "file_path": md_path,
        "title":     title,
        "text":      content,
        "url":       url
    })

# 3.5: Print summary & preview first 3 entries
print(f"Extracted metadata for {len(all_docs)} pages.\n")
for doc in all_docs[:3]:
    print("File Path:", doc["file_path"])
    print("Title    :", doc["title"])
    print("URL      :", doc["url"])
    print("Text (first 200 chars):", doc["text"][:200].replace("\n", " "), "…\n")


Found 1659 markdown files under powerbi-docs/powerbi-docs
Extracted metadata for 1659 pages.

File Path: powerbi-docs/powerbi-docs/paginated-reports/paginated-reports-edit-service.md
Title    : Edit a paginated report from the Power BI service using Report Builder
URL      : https://docs.microsoft.com/en-us/power-bi/paginated-reports/paginated-reports-edit-service
Text (first 200 chars): --- title: "Edit a paginated report from the Power BI service using Report Builder" description: Learn how to edit a paginated report in Power BI Report Builder directly from a workspace in the Power  …

File Path: powerbi-docs/powerbi-docs/paginated-reports/subreports-troubleshoot.md
Title    : Troubleshoot subreports in Power BI paginated reports
URL      : https://docs.microsoft.com/en-us/power-bi/paginated-reports/subreports-troubleshoot
Text (first 200 chars): --- title: "Troubleshoot subreports in Power BI paginated reports" description: Learn about solutions for common issues when using subrepor

Import Stackoverflow

In [ ]:
import requests

def fetch_powerbi_so_questions(pagesize: int = 5, api_key: str = None):
    """
    Retrieve the most recent 'pagesize' questions tagged 'powerbi' on Stack Overflow,
    including 'question_id', 'title', 'accepted_answer_id', and 'link'.
    We will fetch bodies in a separate step.
    """
    url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order":    "desc",
        "sort":     "creation",
        "tagged":   "powerbi",
        "site":     "stackoverflow",
        "pagesize": pagesize
    }
    if api_key:
        params["key"] = api_key

    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json().get("items", [])

# Run with pagesize=5 for a quick test
if __name__ == "__main__":
    so_questions = fetch_powerbi_so_questions(pagesize=5, api_key=None)
    print("Fetched questions (showing up to 5):\n")
    for q in so_questions:
        print(f"Question ID          : {q['question_id']}")
        print(f"Title                : {q['title']}")
        print(f"Accepted Answer ID   : {q.get('accepted_answer_id')}")
        print(f"Link                 : {q.get('link')}")
        print("-" * 80)


Fetched questions (showing up to 5):

Question ID          : 79648862
Title                : Error connecting to Power BI Service from C# to execute XMLA script
Accepted Answer ID   : None
Link                 : https://stackoverflow.com/questions/79648862/error-connecting-to-power-bi-service-from-c-to-execute-xmla-script
--------------------------------------------------------------------------------
Question ID          : 79647944
Title                : Sort Card (new) Visual in Power BI by Rank
Accepted Answer ID   : 79648042
Link                 : https://stackoverflow.com/questions/79647944/sort-card-new-visual-in-power-bi-by-rank
--------------------------------------------------------------------------------
Question ID          : 79647200
Title                : Why does using a relative path with Web.Contents cause sharepoint authentication issues in Power BI
Accepted Answer ID   : None
Link                 : https://stackoverflow.com/questions/79647200/why-does-using-a-relativ

In [ ]:
import requests
from bs4 import BeautifulSoup

# Reuse the previously fetched questions
so_questions = fetch_powerbi_so_questions(pagesize=5, api_key=None)

# 1) Collect all non-None accepted_answer_id values
accepted_ids = [str(q["accepted_answer_id"]) for q in so_questions if q.get("accepted_answer_id")]
if not accepted_ids:
    print("No accepted answers among the fetched questions.")
else:
    print("Accepted Answer IDs to fetch:", accepted_ids)

    # 2) Fetch those answers in one batch
    ids_str = ";".join(accepted_ids)
    answer_url = f"https://api.stackexchange.com/2.3/answers/{ids_str}"
    params = {
        "site": "stackoverflow",
        "filter": "withbody"  # ensures 'body' (HTML) is returned
    }
    response = requests.get(answer_url, params=params)
    response.raise_for_status()
    answer_items = response.json().get("items", [])

    # 3) For each answer, extract the plain text and print a preview
    print(f"\nFetched {len(answer_items)} answers. Preview:\n")
    for ans in answer_items:
        aid       = ans["answer_id"]
        html_body = ans["body"]
        # Convert HTML → plaintext
        text_body = BeautifulSoup(html_body, "html.parser").get_text(separator="\n")
        print(f"Answer ID: {aid}")
        print("Text (first 200 chars):", text_body[:200].replace("\n", " "), "…")
        print("-" * 80)


Accepted Answer IDs to fetch: ['79648042']

Fetched 1 answers. Preview:

Answer ID: 79648042
Text (first 200 chars): I think that a native PBI solution is not available. Here are two workarounds:   The  first workaround  if Manager rank is a column:   Exchange Manager name and Manager rank. This way Manager rank wil …
--------------------------------------------------------------------------------


# Step 4: Chunking Both Official Docs and SO Answers

In [ ]:
# Combined Steps 3 & 4: Fetch SO Answers and Chunk Everything

from bs4 import BeautifulSoup

# --- 3. Fetch the 5 most recent PowerBI SO questions & accepted answers ---

def fetch_powerbi_so_questions(pagesize: int = 5, api_key: str = None):
    url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order":    "desc",
        "sort":     "creation",
        "tagged":   "powerbi",
        "site":     "stackoverflow",
        "pagesize": pagesize
    }
    if api_key:
        params["key"] = api_key
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json().get("items", [])

so_questions = fetch_powerbi_so_questions(pagesize=5, api_key=None)

# Collect accepted_answer_ids
accepted_ids = [str(q["accepted_answer_id"]) for q in so_questions if q.get("accepted_answer_id")]
answer_items = []
if accepted_ids:
    ids_str    = ";".join(accepted_ids)
    ans_url    = f"https://api.stackexchange.com/2.3/answers/{ids_str}"
    params     = {"site": "stackoverflow", "filter": "withbody"}
    resp2      = requests.get(ans_url, params=params)
    resp2.raise_for_status()
    answer_items = resp2.json().get("items", [])
    print(f"Fetched {len(answer_items)} accepted answers from SO.\n")
else:
    print("No accepted answers found among the 5 recent questions.")

# --- 4. Chunking Official Docs and SO Answers ---

# 4.1 Initialize splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 4.2 Chunk official docs (all_docs must already exist)
doc_chunks = []
for doc in all_docs:
    pieces = splitter.split_text(doc["text"])
    for piece in pieces:
        doc_chunks.append({
            "title":  doc["title"],
            "url":    doc["url"],
            "text":   piece,
            "source": "docs"
        })
print(f"Created {len(doc_chunks)} chunks from official docs.")

# 4.3 Chunk SO answers (answer_items)
so_chunks = []
for ans in answer_items:
    aid       = ans["answer_id"]
    qid       = ans["question_id"]
    html_body = ans["body"]
    text_body = BeautifulSoup(html_body, "html.parser").get_text(separator="\n")
    pieces    = splitter.split_text(text_body)
    matching  = next((q for q in so_questions if q["question_id"] == qid), {})
    q_title   = matching.get("title", f"SO Question {qid}")
    q_link    = matching.get("link", f"https://stackoverflow.com/questions/{qid}")
    for piece in pieces:
        so_chunks.append({
            "title":  f"SO: {q_title}",
            "url":    q_link,
            "text":   piece,
            "source": "stackoverflow"
        })
print(f"Created {len(so_chunks)} chunks from StackOverflow answers.")

# 4.4 Combine both sets
all_chunks = doc_chunks + so_chunks
print(f"Total combined chunks: {len(all_chunks)}")

# 4.5 Preview first 2 combined chunks
for i, chunk in enumerate(all_chunks[:2], 1):
    print(f"\n--- Chunk {i} ---")
    print("Source :", chunk["source"])
    print("Title  :", chunk["title"])
    print("URL    :", chunk["url"])
    print("Text   :", chunk["text"][:200].replace('\n',' '), "…")


Fetched 1 accepted answers from SO.

Created 41801 chunks from official docs.
Created 3 chunks from StackOverflow answers.
Total combined chunks: 41804

--- Chunk 1 ---
Source : docs
Title  : Edit a paginated report from the Power BI service using Report Builder
URL    : https://docs.microsoft.com/en-us/power-bi/paginated-reports/paginated-reports-edit-service
Text   : --- title: "Edit a paginated report from the Power BI service using Report Builder" description: Learn how to edit a paginated report in Power BI Report Builder directly from a workspace in the Power  …

--- Chunk 2 ---
Source : docs
Title  : Edit a paginated report from the Power BI service using Report Builder
URL    : https://docs.microsoft.com/en-us/power-bi/paginated-reports/paginated-reports-edit-service
Text   : [!INCLUDE [applies-yes-report-builder-no-desktop](../includes/applies-yes-report-builder-no-desktop.md)]   This article explains how to quickly edit a paginated report in Power BI Report Builder direc …


In [ ]:
# ----------------------------------------
# Step 5 (MPNet on CPU with Progress Logging & Split Encoding)
# ----------------------------------------

import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import torch

# Ensure DEVICE is set (should be "cpu" in this runtime)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device for embedding:", DEVICE)

# Assume `doc_chunks` and `so_chunks` are already defined lists of dicts, each with a "text" field.
# For example:
# doc_chunks = [{"title": ..., "url": ..., "text": ..., "source": "docs"}, ...]
# so_chunks  = [{"title": ..., "url": ..., "text": ..., "source": "stackoverflow"}, ...]

# 5.1: Load MPNet embedder on CPU (will be slow, but we add logging)
embed_model = SentenceTransformer("all-mpnet-base-v2", device=DEVICE)

# 5.2: Encode `doc_chunks` first, in batches, with progress logging
doc_texts = [chunk["text"] for chunk in doc_chunks]
num_doc = len(doc_texts)
batch_size = 128
embeddings_docs = []

print(f"\nEncoding {num_doc} document chunks with MPNet (batch_size={batch_size}):")
total_doc_batches = (num_doc + batch_size - 1) // batch_size

for batch_idx in range(total_doc_batches):
    start = batch_idx * batch_size
    end   = min(start + batch_size, num_doc)
    batch_texts = doc_texts[start:end]

    # encode returns a numpy array when convert_to_numpy=True
    batch_emb = embed_model.encode(batch_texts, convert_to_numpy=True)
    embeddings_docs.append(batch_emb.astype("float32"))

    print(f"  Docs batch {batch_idx+1}/{total_doc_batches} "
          f"({end}/{num_doc} chunks encoded)")

embeddings_docs = np.vstack(embeddings_docs)  # shape: (num_doc_chunks, 768)
print(f"✅ Completed MPNet encoding for docs: shape {embeddings_docs.shape}")

# 5.3: Encode `so_chunks` next, in batches, with progress logging
so_texts = [chunk["text"] for chunk in so_chunks]
num_so = len(so_texts)
embeddings_so = []

print(f"\nEncoding {num_so} StackOverflow chunks with MPNet (batch_size={batch_size}):")
total_so_batches = (num_so + batch_size - 1) // batch_size

for batch_idx in range(total_so_batches):
    start = batch_idx * batch_size
    end   = min(start + batch_size, num_so)
    batch_texts = so_texts[start:end]

    batch_emb = embed_model.encode(batch_texts, convert_to_numpy=True)
    embeddings_so.append(batch_emb.astype("float32"))

    print(f"  SO batch {batch_idx+1}/{total_so_batches} "
          f"({end}/{num_so} chunks encoded)")

embeddings_so = np.vstack(embeddings_so)  # shape: (num_so_chunks, 768)
print(f"✅ Completed MPNet encoding for SO: shape {embeddings_so.shape}")

# 5.4: Combine doc and SO embeddings into one array
embeddings = np.vstack([embeddings_docs, embeddings_so])
print(f"\n✅ Combined embeddings shape: {embeddings.shape}  (docs + SO)")

# 5.5: Build a new FAISS index over these 768-dim embeddings
dim = embeddings.shape[1]  # should be 768
index_mpnet = faiss.IndexFlatL2(dim)
index_mpnet.add(embeddings)
print(f"✅ FAISS index built with {index_mpnet.ntotal} vectors (MPNet)")

# 5.6: Persist index and chunk metadata for later retrieval
faiss.write_index(index_mpnet, "powerbi_rag_index_mpnet.faiss")
with open("powerbi_rag_docs_mpnet.pkl", "wb") as f:
    # We need to recreate `all_chunks` in the same order as embeddings:
    # all_chunks = doc_chunks + so_chunks
    all_chunks = doc_chunks + so_chunks
    pickle.dump(all_chunks, f)

print("Saved 'powerbi_rag_index_mpnet.faiss' and 'powerbi_rag_docs_mpnet.pkl'\n")


Using device for embedding: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Encoding 41801 document chunks with MPNet (batch_size=128):
  Docs batch 1/327 (128/41801 chunks encoded)
  Docs batch 2/327 (256/41801 chunks encoded)
  Docs batch 3/327 (384/41801 chunks encoded)
  Docs batch 4/327 (512/41801 chunks encoded)
  Docs batch 5/327 (640/41801 chunks encoded)
  Docs batch 6/327 (768/41801 chunks encoded)
  Docs batch 7/327 (896/41801 chunks encoded)
  Docs batch 8/327 (1024/41801 chunks encoded)
  Docs batch 9/327 (1152/41801 chunks encoded)
  Docs batch 10/327 (1280/41801 chunks encoded)
  Docs batch 11/327 (1408/41801 chunks encoded)
  Docs batch 12/327 (1536/41801 chunks encoded)
  Docs batch 13/327 (1664/41801 chunks encoded)
  Docs batch 14/327 (1792/41801 chunks encoded)
  Docs batch 15/327 (1920/41801 chunks encoded)
  Docs batch 16/327 (2048/41801 chunks encoded)
  Docs batch 17/327 (2176/41801 chunks encoded)
  Docs batch 18/327 (2304/41801 chunks encoded)
  Docs batch 19/327 (2432/41801 chunks encoded)
  Docs batch 20/327 (2560/41801 chunks enco

In [ ]:
# ----------------------------------------
# Step 6 (updated): Load MPNet Index & Define Retrieval with rewrite_query()
# ----------------------------------------

import faiss
import pickle
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 6.1: Reload the MPNet FAISS index and chunk metadata from disk
with open("powerbi_rag_docs_mpnet.pkl", "rb") as f:
    all_chunks_mpnet = pickle.load(f)

index_mpnet = faiss.read_index("powerbi_rag_index_mpnet.faiss")

# 6.2: Re‐initialize the MPNet embedder (same as in Step 5)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = SentenceTransformer("all-mpnet-base-v2", device=DEVICE)

# 6.3: Load a small T5 model & tokenizer for query rewriting
rewrite_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
rewrite_model     = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(DEVICE)

# 6.4: Define rewrite_query() to emphasize “Merge Queries (desktop-merge-queries)”
def rewrite_query(raw_query: str) -> str:
    """
    Rephrase the user’s Power BI question so it explicitly asks how to merge two tables
    using the Power Query Editor’s Merge Queries feature (desktop-merge-queries).
    """
    prompt = (
        "Rewrite this Power BI question so it explicitly asks how to merge two tables "
        "using the Power Query Editor’s Merge Queries (desktop-merge-queries) page.\n"
        f"Original question: {raw_query}\n"
        "Rewritten:"
    )
    inputs = rewrite_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(DEVICE)

    outputs = rewrite_model.generate(
        **inputs,
        max_new_tokens=50,
        num_beams=5,
        do_sample=False,
        eos_token_id=rewrite_tokenizer.eos_token_id
    )
    rewritten = rewrite_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return rewritten

# 6.5: Define retrieve() to return top-k chunks from the MPNet index
def retrieve(query: str, k: int = 1) -> list[dict]:
    """
    1. Rewrite the raw query via rewrite_query().
    2. Embed the rewritten text with MPNet and search FAISS for top k.
    3. Return the corresponding chunk dicts from all_chunks_mpnet.
    """
    # (a) Rewrite the user’s question for better “Merge Queries” matching
    rewritten_q = rewrite_query(query)

    # (b) Embed the rewritten query with MPNet
    q_emb = embed_model.encode([rewritten_q], convert_to_numpy=True)

    # (c) FAISS search: retrieve top-k indices
    distances, indices = index_mpnet.search(q_emb, k)

    # (d) Return those chunk dict(s)
    return [all_chunks_mpnet[i] for i in indices[0]]

# 6.6: Quick retrieval test (run this to confirm your retrieve() works)
if __name__ == "__main__":
    sample = retrieve("How can I merge two tables in Power BI Desktop?", k=3)
    print("\nRetrieved chunks:")
    for c in sample:
        print(" •", c["title"], "→", c["url"])
        print("   ", c["text"][:100].replace("\n", " "), "…\n")



Retrieved chunks:
 • Tutorial: Shape and combine data in Power BI Desktop → https://docs.microsoft.com/en-us/power-bi/connect-data/desktop-shape-and-combine-data
    1. Select **OK**.     Power Query Editor creates a new column at the end of the query, which contain …

 • Tutorial: Shape and combine data in Power BI Desktop → https://docs.microsoft.com/en-us/power-bi/connect-data/desktop-shape-and-combine-data
    The **Merge** window appears. It prompts you to select which table you'd like merged into the select …

 • Tutorial: Shape and combine data in Power BI Desktop → https://docs.microsoft.com/en-us/power-bi/connect-data/desktop-shape-and-combine-data
    :::image type="content" source="media/desktop-shape-and-combine-data/merge-queries.png" alt-text="Sc …



In [ ]:
# ----------------------------------------
# Step 8: Generation with Stronger Prompt (Flan‐T5‐Large)
# ----------------------------------------

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# (Assumes DEVICE, retrieve(), and pick_relevant_url() are already defined.)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device for generation:", DEVICE)

# 8.1 Load Flan-T5-Large for generation
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model     = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(DEVICE)

# 8.2 Build a “stronger” prompt that enforces one full sentence + URL
def build_powerbi_prompt(query: str, chunks: list[dict]) -> str:
    """
    - List each retrieved chunk as "Title (URL): snippet…"
    - Then instruct:
      "Using ONLY the context below, write one clear, complete sentence that directly tells
       the user exactly how to do the task in Power BI Desktop. Append exactly one URL at the end.
       Do not respond with only a title, fragment, or URL—write one full sentence."
    """
    context_parts = []
    for c in chunks:
        snippet = c["text"].replace("\n", " ")
        snippet = snippet[:150] + ("…" if len(snippet) > 150 else "")
        context_parts.append(f"{c['title']} ({c['url']}): {snippet}")
    context_str = "\n\n".join(context_parts)

    return (
        "You are a Power BI expert. Use ONLY the context below (from official docs & StackOverflow) "
        "to write one clear, complete sentence that directly tells the user how to perform their requested task in Power BI Desktop. "
        "Then append exactly one URL at the end. Do not respond with only a title, fragment, or URL—write one full sentence.\n\n"
        f"{context_str}\n\n"
        f"User’s question: {query}\n\n"
        "Answer (one sentence + URL):"
    )

# 8.3 Helper to pick a single citation URL
def pick_relevant_url(chunks: list[dict], query: str) -> str:
    lower_q = query.lower()
    for c in chunks:
        url_lower = c["url"].lower()
        for term in ["dax", "transform-model", "visual", "chart", "merge", "filter", "measure"]:
            if term in url_lower and term in lower_q:
                return c["url"]
    return chunks[0]["url"]

# 8.4 Generation function using the stronger prompt
def generate_powerbi_response(query: str, k: int = 5) -> str:
    # A) Retrieve top-k chunks
    chunks = retrieve(query, k)

    # B) Debug: print retrieved chunk titles + URLs
    print("Retrieved chunks:")
    for c in chunks:
        print(" •", c["title"], "→", c["url"])
    print("-----\n")

    # C) Build the stronger prompt
    prompt_text = build_powerbi_prompt(query, chunks)
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(DEVICE)

    # D) Generate an answer with beam search
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        num_beams=5,
        early_stopping=True
    )
    draft = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # E) If draft is empty or whitespace, return a URL fallback
    if not draft:
        fallback_url = pick_relevant_url(chunks, query)
        return f"Please refer to {fallback_url} for more details."

    # F) If draft contains “http” or starts with “http”, accept it as-is
    if draft.lower().startswith("http") or "https://" in draft:
        return draft

    # G) Otherwise, append a citation URL at the end
    relevant_url = pick_relevant_url(chunks, query)
    return f"{draft}\n\nFor more details, see {relevant_url}."

# 8.5 Quick generation test
if __name__ == "__main__":
    test_query = "How do I create a bar chart in Power BI?"
    response   = generate_powerbi_response(test_query, k=5)
    print("---- Generated Answer ----\n", response)


Using device for generation: cpu
Retrieved chunks:
 • Tutorial: Build a bar chart → https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart
 • Tutorial: Build a bar chart → https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart
 • Charts in Power BI paginated reports (Power BI Report Builder) → https://docs.microsoft.com/en-us/power-bi/paginated-reports/report-design/visualizations/charts-report-builder
 • Format axis labels on a paginated report chart (Power BI Report Builder) → https://docs.microsoft.com/en-us/power-bi/paginated-reports/report-design/visualizations/format-axis-labels-chart-report-builder
 • Create a column chart in Power BI → https://docs.microsoft.com/en-us/power-bi/visuals/power-bi-visualization-column-charts
-----



In [ ]:
# ----------------------------------------
# Step 9: Simple Chatbot Loop
# ----------------------------------------

# (Assumes DEVICE, retrieve(), pick_relevant_url(),
#  and generate_powerbi_response() are already defined.)

def powerbi_chatbot():
    print("Power BI Help Chatbot (type 'exit' or blank to quit)\n")
    while True:
        user_query = input("You: ").strip()
        if not user_query or user_query.lower() == "exit":
            print("Goodbye!")
            break

        # Generate and display answer
        answer = generate_powerbi_response(user_query, k=5)
        print("\nAssistant:", answer, "\n")

# Run the chatbot
if __name__ == "__main__":
    powerbi_chatbot()


Power BI Help Chatbot (type 'exit' or blank to quit)

You: how to create bar chat
Retrieved chunks:
 • Chat in Microsoft Teams directly from the Power BI service → https://docs.microsoft.com/en-us/power-bi/collaborate-share/service-share-report-teams
 • Semantic model details → https://docs.microsoft.com/en-us/power-bi/connect-data/service-dataset-details-page
 • Lead data-driven discussions in Microsoft Teams → https://docs.microsoft.com/en-us/power-bi/consumer/business-user-teams-share-data
 • Chat in Microsoft Teams directly from the Power BI service → https://docs.microsoft.com/en-us/power-bi/collaborate-share/service-share-report-teams
 • Analyze your Teams collaboration data → https://docs.microsoft.com/en-us/power-bi/consumer/business-user-teams-collaboration-data
-----


Assistant: You can chat in Microsoft Teams directly from the Power BI service.

For more details, see https://docs.microsoft.com/en-us/power-bi/collaborate-share/service-share-report-teams. 

You: how to create

In [ ]:
# ----------------------------------------
# Step 10: Evaluation & Logging
# ----------------------------------------

import pandas as pd
import numpy as np
import csv
import os

# (Assumes retrieve(), generate_powerbi_response(), pick_relevant_url() are already defined.)

# 10.1: Define a small gold‐standard test set for retrieval
#       Each entry is a tuple: (query, [list of known relevant URLs])
gold_retrieval = [
    (
        "How do I create a bar chart in Power BI?",
        ["https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart"]
    ),
    (
        "How can I merge two tables in Power BI Desktop?",
        ["https://docs.microsoft.com/en-us/power-bi/connect-data/desktop-shape-and-combine-data"]
    ),
    (
        "How to write a DAX measure for year-to-date sales?",
        ["https://docs.microsoft.com/en-us/power-bi/transform-model/desktop-quickstart-learn-dax-basics"]
    ),
    # Add more query → [relevant URLs] as needed
]

# 10.2: Retrieval metrics: Recall@k and MRR
def evaluate_retrieval(test_set, k=5):
    """
    For each (query, relevant_urls) in test_set:
      1) Run retrieve(query, k) → list of chunks
      2) Extract chunk["url"] for each retrieved chunk
      3) Compute if any relevant URL appears in top‐k (Recall@k)
      4) Compute reciprocal rank: 1 / (rank of first relevant URL), else 0
    Returns a DataFrame summarizing each query’s Recall@k and RR; and
    overall averages.
    """
    rows = []
    rr_list = []
    recall_list = []
    for query, relevant_urls in test_set:
        retrieved_chunks = retrieve(query, k)
        retrieved_urls = [c["url"] for c in retrieved_chunks]

        # Compute Recall@k: 1 if any of relevant_urls ∈ retrieved_urls, else 0
        hit = int(any(url in retrieved_urls for url in relevant_urls))
        recall_list.append(hit)

        # Compute Reciprocal Rank (RR)
        rr = 0.0
        for rank, url in enumerate(retrieved_urls, start=1):
            if url in relevant_urls:
                rr = 1.0 / rank
                break
        rr_list.append(rr)

        rows.append({
            "query": query,
            "relevant_urls": ", ".join(relevant_urls),
            "retrieved_urls": ", ".join(retrieved_urls),
            f"recall@{k}": hit,
            "reciprocal_rank": rr
        })

    df = pd.DataFrame(rows)
    overall = {
        f"mean_recall@{k}": np.mean(recall_list),
        "mean_MRR": np.mean(rr_list)
    }
    return df, overall

# 10.3: Run retrieval evaluation and save to CSV
if __name__ == "__main__":
    retrieval_df, retrieval_overall = evaluate_retrieval(gold_retrieval, k=5)
    print("\nRetrieval results:")
    print(retrieval_df)
    print("\nOverall metrics:", retrieval_overall)

    # Save per-query results
    retrieval_df.to_csv("retrieval_evaluation.csv", index=False)

    # Append overall metrics to a separate CSV or JSON
    with open("retrieval_overall_metrics.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=list(retrieval_overall.keys()))
        writer.writeheader()
        writer.writerow(retrieval_overall)

    print("\nSaved retrieval_evaluation.csv and retrieval_overall_metrics.csv")

# 10.4: Generation logging (interactive / batch)
#       We’ll record: query, retrieved URLs, generated answer.
#       Use the same test set, or interactively log user sessions.

def log_generation(query, chunks, answer, filename="generation_log.csv"):
    """
    Append one row to `generation_log.csv`, with columns:
      query, retrieved_urls, answer
    If the file does not exist, create it and write headers.
    """
    retrieved_urls = "; ".join(c["url"] for c in chunks)
    row = {
        "query": query,
        "retrieved_urls": retrieved_urls,
        "answer": answer
    }
    file_exists = os.path.isfile(filename)
    with open(filename, "a", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["query", "retrieved_urls", "answer"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

# Example: batch‐evaluate generation on gold_retrieval queries
if __name__ == "__main__":
    # Remove any existing log
    if os.path.isfile("generation_log.csv"):
        os.remove("generation_log.csv")
    print("\nLogging generation results:")
    for query, _ in gold_retrieval:
        chunks = retrieve(query, k=5)
        answer = generate_powerbi_response(query, k=5)
        print(f"Query: {query}\nAnswer: {answer}\n")
        log_generation(query, chunks, answer, filename="generation_log.csv")
    print("Saved generation_log.csv")

# 10.5: Load and display logs (optional)
if __name__ == "__main__":
    if os.path.isfile("generation_log.csv"):
        df_gen = pd.read_csv("generation_log.csv")
        print("\nGeneration Log Preview:")
        print(df_gen.head())



Retrieval results:
                                               query  \
0           How do I create a bar chart in Power BI?   
1    How can I merge two tables in Power BI Desktop?   
2  How to write a DAX measure for year-to-date sa...   

                                       relevant_urls  \
0  https://docs.microsoft.com/en-us/power-bi/deve...   
1  https://docs.microsoft.com/en-us/power-bi/conn...   
2  https://docs.microsoft.com/en-us/power-bi/tran...   

                                      retrieved_urls  recall@5  \
0  https://docs.microsoft.com/en-us/power-bi/deve...         1   
1  https://docs.microsoft.com/en-us/power-bi/conn...         1   
2  https://docs.microsoft.com/en-us/power-bi/crea...         0   

   reciprocal_rank  
0              1.0  
1              1.0  
2              0.0  

Overall metrics: {'mean_recall@5': np.float64(0.6666666666666666), 'mean_MRR': np.float64(0.6666666666666666)}

Saved retrieval_evaluation.csv and retrieval_overall_metrics.csv

Lo